In [8]:
import numpy as np
import json
import os
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.utils import resample
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import brier_score_loss
from sklearn.metrics import balanced_accuracy_score
from sklearn.feature_selection import chi2
from sklearn.model_selection import GridSearchCV, LeaveOneOut, StratifiedKFold

In [2]:
with open(r'C:\Users\Axel\Desktop\projet stat\bdd\liste_texte_oui.txt', 'r') as f:
    liste_texte_oui = f.read().splitlines()
    
with open(r'C:\Users\Axel\Desktop\projet stat\bdd\liste_question.txt', 'r') as f:
    liste_question = f.read().splitlines()
    
data_train_clean= pd.read_csv(r'C:\Users\Axel\Desktop\projet stat\bdd\data_train_clean.csv')
data_train_clean.set_index('node_id',inplace=True)

In [3]:
len(liste_texte_oui), len(liste_question),len(data_train_clean)

(1488, 775, 423)

In [4]:
#dico pour stocker les parametre opti pour une random forest
dico_param_random_forest=dict.fromkeys(liste_texte_oui,None)

#dico pour stocker les scores de différentes métrique sur l'echantillon de train (équilibré)
dico_accurency_train_random_forest=dict.fromkeys(liste_texte_oui,None)
dico_recall_train_random_forest=dict.fromkeys(liste_texte_oui,None)
dico_prec_AUC_train_random_forest=dict.fromkeys(liste_texte_oui,None)
dico_brier_train_random_forest=dict.fromkeys(liste_texte_oui,None)


#dico pour stocker les scores de différentes métrique sur l'echantillon de test
dico_accurency_test_random_forest=dict.fromkeys(liste_texte_oui,None)
dico_recall_test_random_forest=dict.fromkeys(liste_texte_oui,None)
dico_prec_AUC_test_random_forest=dict.fromkeys(liste_texte_oui,None)
dico_brier_test_random_forest=dict.fromkeys(liste_texte_oui,None)


#dico pour stocker les scores de différentes métrique sur l'echantillon de train (équilibré) apès calibration
dico_accurency_train_cali_random_forest=dict.fromkeys(liste_texte_oui,None)
dico_recall_train_cali_random_forest=dict.fromkeys(liste_texte_oui,None)
dico_prec_AUC_train_cali_random_forest=dict.fromkeys(liste_texte_oui,None)
dico_brier_train_cali_random_forest=dict.fromkeys(liste_texte_oui,None)

#dico pour stocker les scores de différentes métrique sur l'echantillon de test apès calibration
dico_accurency_test_cali_random_forest=dict.fromkeys(liste_texte_oui,None)
dico_recall_test_cali_random_forest=dict.fromkeys(liste_texte_oui,None)
dico_prec_AUC_test_cali_random_forest=dict.fromkeys(liste_texte_oui,None)
dico_brier_test_cali_random_forest=dict.fromkeys(liste_texte_oui,None)

In [5]:
#A noter: liste texte_oui et liste_question doivent etre défini précédement
def equilibre(data,i):
    if isinstance(i, str):
        text=i
    else: 
        text = liste_texte_oui[i]
    
    #creation base
    colonnes = list(liste_question)
    colonnes.append(text)
    base_text = data[list(colonnes)]
    
    #creation test set et train set
    X_train, X_test, Y_train, Y_test = train_test_split(base_text.iloc[:,0:-1], base_text[text],stratify=base_text[text], test_size=0.2,random_state=10)    
    
    # Returning to one dataframe
    training_set = pd.concat([X_train, Y_train], axis=1)
    
    # Separating classes
    class1 = training_set[training_set[text] == 1]
    class0 = training_set[training_set[text] == 0]
    
    #taille des classes
    n0=len(class0)
    n1=len(class1)
    
    if n0 > n1 :
        undersample = resample(class0, replace=True, n_samples=len(class1), random_state=10)
        undersample_train = pd.concat([class1, undersample])
        x_train = undersample_train.drop(text, axis=1)
        y_train = undersample_train[text]
        return (x_train, X_test, y_train, Y_test)
    elif n0 < n1 :
        undersample = resample(class1, replace=True, n_samples=len(class0), random_state=10)
        undersample_train = pd.concat([class0, undersample])
        x_train = undersample_train.drop(text, axis=1)
        y_train = undersample_train[text]
        return (x_train, X_test, y_train, Y_test)
    else:
        return (X_train, X_test, Y_train, Y_test)
    

In [6]:
#boucle d'entrainement des textes
compteur=0
for texte in liste_texte_oui :
    compteur=compteur+1
    print(compteur)
    
    #on équilibre les échantillons
    x_train, X_test, y_train, Y_test=equilibre(data_train_clean,texte)
    
    #on print le premier client du x_train pour vérifier qu'on a bien tjr les mêmes
    print(x_train.index[0])
    
    #on défini les meilleurs parametres sur le modèle
    model=RandomForestClassifier(max_features='auto',random_state=0,n_jobs=-1)
    param={'n_estimators':[50,100,200],'max_depth':[3,5,9]}
    
    #on selectionne les meilleurs param par validation croisée type stratifié car on souhaite donner une meme importance 
    #(ne change par grand chose par rapport au Kfold calssique dans notre cas)
    grid=GridSearchCV(model,param,cv=StratifiedKFold(n_splits=5, shuffle=True,random_state=0),scoring='average_precision',n_jobs=-1)
    grid.fit(x_train,y_train)
    
    #on stocke les meilleurs param
    dico_param_random_forest[texte]=grid.best_params_
    
    #on defini ce meilleur estimateur
    model1=grid.best_estimator_
    
    #on stocke les différents scores de ce meilleur estimateur
    pred_train=model1.predict(x_train)
    pred_test=model1.predict(X_test)
    
    pred_proba_train=model1.predict_proba(x_train)[:,1]
    pred_proba_test=model1.predict_proba(X_test)[:,1]
    
    #Sur les dico du train
    
    dico_accurency_train_random_forest[texte]=balanced_accuracy_score(y_train,pred_train)
    dico_recall_train_random_forest[texte]=recall_score(y_train,pred_train)
    dico_prec_AUC_train_random_forest[texte]=average_precision_score(y_train,pred_proba_train)
    dico_brier_train_random_forest[texte]=brier_score_loss(y_train,pred_proba_train)
    
    #Sur les dico du test
    
    dico_accurency_test_random_forest[texte]=balanced_accuracy_score(Y_test,pred_test)
    dico_recall_test_random_forest[texte]=recall_score(Y_test,pred_test)
    dico_prec_AUC_test_random_forest[texte]=average_precision_score(Y_test,pred_proba_test)
    dico_brier_test_random_forest[texte]=brier_score_loss(Y_test,pred_proba_test)
    
    #On calibre le modele obtenu
    model2=RandomForestClassifier(n_estimators=dico_param_random_forest[texte]['n_estimators'],max_depth=dico_param_random_forest[texte]['max_depth'],max_features='auto',random_state=0,n_jobs=-1)
    calibre = CalibratedClassifierCV(model2, cv=StratifiedKFold(n_splits=5, shuffle=True,random_state=0), method='sigmoid')

    calibre.fit(x_train, y_train) #The final estimator is an ensemble of n_cv fitted classifer and calibrator pairs, where n_cv is the number of cross-validation folds. The output is the average predicted probabilities of all pairs.
    
    #on stocke les différents scores de ce meilleur estimateur
    calibre_pred_train=calibre.predict(x_train)
    calibre_pred_test=calibre.predict(X_test)
    calibrated_probs_train = calibre.predict_proba(x_train)[:,1]
    calibrated_probs_test = calibre.predict_proba(X_test)[:,1]
    
    #Sur les dico du train
    
    dico_accurency_train_cali_random_forest[texte]=balanced_accuracy_score(y_train,calibre_pred_train)
    dico_recall_train_cali_random_forest[texte]=recall_score(y_train,calibre_pred_train)
    dico_prec_AUC_train_cali_random_forest[texte]=average_precision_score(y_train,calibrated_probs_train)
    dico_brier_train_cali_random_forest[texte]=brier_score_loss(y_train,calibrated_probs_train)
    
    
    #Sur les dico du test
    
    dico_accurency_test_cali_random_forest[texte]=balanced_accuracy_score(Y_test,calibre_pred_test)
    dico_recall_test_cali_random_forest[texte]=recall_score(Y_test,calibre_pred_test)
    dico_prec_AUC_test_cali_random_forest[texte]=average_precision_score(Y_test,calibrated_probs_test)
    dico_brier_test_cali_random_forest[texte]=brier_score_loss(Y_test,calibrated_probs_test)

1
d37394c52f494d90012f4dd10c7a240a
2
d37394c523fb2004012409fb9a243796
3
d3739445355c87aa013562a523fb0bcd
4
d37394c52ce75667012cff26d6797e61
5
d37394c523fb2004012409f34bce36d3
6
2c90f5884c600d3f014c984ba1bd6081
7
d37394c52f494d90012f4de129f42448
8
d37394c52e1a2820012e28c216e43313
9
d37394c52f494d90012f4dfb5b0c2530
10
d37394c52dba36fd012dec6837d5427a
11
d373944536c784aa01371874fe756921
12
d37394c520f380a20121872cb6bb3962
13
2c90f5884c600d3f014c984ba1bd6081
14
d37394c51fa9e652011fec25181e39cd
15
d37394c52f494d90012f4dd16c322413
16
d37394453e1e8ab0013e30fd6b7f2dca
17
d37394c52f494d90012f4de129f42448
18
d37394c523fb2004012409facc223784
19
d37394c52f494d90012f4e010e5d25a4
20
d37394c52f494d90012f4e0093f52596
21
d37394c52f494d90012f4de1a73b2453
22
4028807d16245e4d011624789bd97b1a
23
d37394c5208c61e801208f987ca5085f
24
d37394c52f494d90012f4dfd89482574
25
d37394c52f494d90012f4dfd1d342568
26
d37394c52f494d90012f4dfb5b0c2530
27
d37394c52f494d90012f4dfb5b0c2530
28
2c90f58959f683cf0159fa24849062a5
2

In [9]:
#on enregistre les dictionnaires
#dico_param_random_forest
with open(r'C:\Users\Axel\Desktop\projet stat\dico_random_forest\dico_param_random_forest.json', 'w') as fp:
    json.dump(dico_param_random_forest, fp,  indent=2)


In [10]:
#dico train base

#dico_accurency_train_random_forest
with open(r'C:\Users\Axel\Desktop\projet stat\dico_random_forest\dico_accurency_train_random_forest.json', 'w') as fp:
    json.dump(dico_accurency_train_random_forest, fp,  indent=2)

#dico_recall_train_random_forest
with open(r'C:\Users\Axel\Desktop\projet stat\dico_random_forest\dico_recall_train_random_forest.json', 'w') as fp:
    json.dump(dico_recall_train_random_forest, fp,  indent=2)

    
#dico_prec_AUC_train_random_forest    
with open(r'C:\Users\Axel\Desktop\projet stat\dico_random_forest\dico_prec_AUC_train_random_forest.json', 'w') as fp:
    json.dump(dico_prec_AUC_train_random_forest, fp,  indent=2)

    
#dico_brier_train_random_forest
with open(r'C:\Users\Axel\Desktop\projet stat\dico_random_forest\dico_brier_train_random_forest.json', 'w') as fp:
    json.dump(dico_brier_train_random_forest, fp,  indent=2)

In [11]:
#dico test base

#dico_accurency_test_random_forest
with open(r'C:\Users\Axel\Desktop\projet stat\dico_random_forest\dico_accurency_test_random_forest.json', 'w') as fp:
    json.dump(dico_accurency_test_random_forest, fp,  indent=2)
 
    #dico_recall_test_random_forest
with open(r'C:\Users\Axel\Desktop\projet stat\dico_random_forest\dico_recall_test_random_forest.json', 'w') as fp:
    json.dump(dico_recall_test_random_forest, fp,  indent=2)

    
#dico_prec_AUC_test_random_forest
with open(r'C:\Users\Axel\Desktop\projet stat\dico_random_forest\dico_prec_AUC_test_random_forest.json', 'w') as fp:
    json.dump(dico_prec_AUC_test_random_forest, fp,  indent=2)

    
#dico_brier_test_random_forest
with open(r'C:\Users\Axel\Desktop\projet stat\dico_random_forest\dico_brier_test_random_forest.json', 'w') as fp:
    json.dump(dico_brier_test_random_forest, fp,  indent=2)


In [12]:
#dico train cali


#dico_accurency_train_cali_random_forest
with open(r'C:\Users\Axel\Desktop\projet stat\dico_random_forest\dico_accurency_train_cali_random_forest.json', 'w') as fp:
    json.dump(dico_accurency_train_cali_random_forest, fp,  indent=2)

    
#dico_recall_train_cali_random_forest    
with open(r'C:\Users\Axel\Desktop\projet stat\dico_random_forest\dico_recall_train_cali_random_forest.json', 'w') as fp:
    json.dump(dico_recall_train_cali_random_forest, fp,  indent=2)
  

    #dico_prec_AUC_train_cali_random_forest
with open(r'C:\Users\Axel\Desktop\projet stat\dico_random_forest\dico_prec_AUC_train_cali_random_forest.json', 'w') as fp:
    json.dump(dico_prec_AUC_train_cali_random_forest, fp,  indent=2)

    
#dico_brier_train_cali_random_forest
with open(r'C:\Users\Axel\Desktop\projet stat\dico_random_forest\dico_brier_train_cali_random_forest.json', 'w') as fp:
    json.dump(dico_brier_train_cali_random_forest, fp,  indent=2)

In [13]:
#dico test cali

#dico_accurency_test_cali_random_forest
with open(r'C:\Users\Axel\Desktop\projet stat\dico_random_forest\dico_accurency_test_cali_random_forest.json', 'w') as fp:
    json.dump(dico_accurency_test_cali_random_forest, fp,  indent=2)
    
#dico_recall_test_cali_random_forest
with open(r'C:\Users\Axel\Desktop\projet stat\dico_random_forest\dico_recall_test_cali_random_forest.json', 'w') as fp:
    json.dump(dico_recall_test_cali_random_forest, fp,  indent=2)
    
#dico_prec_AUC_test_cali_random_forest
with open(r'C:\Users\Axel\Desktop\projet stat\dico_random_forest\dico_prec_AUC_test_cali_random_forest.json', 'w') as fp:
    json.dump(dico_prec_AUC_test_cali_random_forest, fp,  indent=2)
    
#dico_brier_test_cali_random_forest
with open(r'C:\Users\Axel\Desktop\projet stat\dico_random_forest\dico_brier_test_cali_random_forest.json', 'w') as fp:
    json.dump(dico_brier_test_cali_random_forest, fp,  indent=2)

In [22]:
pd.array(list(dico_brier_test_cali_random_forest.values())).mean()

0.18573951895237206